In [ ]:
# !pip install linearmodels

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.panel import PanelOLS
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import os
import textwrap

# ==========================================
# 1. SETUP & FEATURE ENGINEERING
# ==========================================
# UPDATE!!!: Local Directory Paths
print(">> PHASE 4: INITIALIZING DATA...")

# CONFIGURATION
DATA_DIR = r'data/raw'
OUTPUT_DIR = r'data/processed'
IMG_DIR = r'data/visuals'

for d in [OUTPUT_DIR, IMG_DIR]:
    if not os.path.exists(d):
        os.makedirs(d)

# Load Artifacts
df_raw = pd.read_csv(os.path.join(OUTPUT_DIR, 'ERI_Phase2_Track1_Model.csv'))
df_eri = pd.read_csv(os.path.join(OUTPUT_DIR, 'ERI_Phase3_Analytical_Base.csv'))
df_meta = pd.read_csv(os.path.join(OUTPUT_DIR, 'ERI_Phase3_Sector_Archetypes.csv'))

# --- CRITICAL FIX 1: FORCE COLUMN RENAMING ---
# Ensure the column is named 'Archetype'
if 'Archetype' not in df_meta.columns:
    candidates = ['Resilience_Archetype', 'resilience_archetype', 'archetype', 'Archetypes']
    for c in candidates:
        if c in df_meta.columns:
            df_meta.rename(columns={c: 'Archetype'}, inplace=True)
            break
    if 'Archetype' not in df_meta.columns:
        df_meta.rename(columns={df_meta.columns[-1]: 'Archetype'}, inplace=True)

# --- CRITICAL FIX 2: TRANSLATE LABELS ---
# Map the "Technical" names to the "User Preferred" names
label_map = {
    'Stalwart (High Res, Low Vol)': 'All-Weather Star\n(High Res, Low Vol)',
    'Cyclical Grower (High Res, High Vol)': 'Volatile Grower\n(High Res, High Vol)',
    'Stagnant (Low Res, Low Vol)': 'Safe Stagnator\n(Low Res, Low Vol)',
    'Distressed (Low Res, High Vol)': 'Distressed\n(Low Res, High Vol)'
}

# Apply mapping (If the key isn't found, it keeps the original value)
df_meta['Archetype'] = df_meta['Archetype'].map(lambda x: label_map.get(x, x))
df_meta['Archetype'] = df_meta['Archetype'].fillna('Distressed\n(Low Res, High Vol)')

# --- A. Construct Labor Productivity Proxy ---
file_gdp = 'gdp_current_prices_by_industry.csv'
file_emp = 'DOS_Employment_Change_By_Sector_Quarterly.csv'

df_gdp = df_raw[df_raw['source_file'] == file_gdp][['date', 'clean_sector', 'value']].rename(columns={'value': 'GDP_Nominal'})
df_emp = df_raw[df_raw['source_file'] == file_emp][['date', 'clean_sector', 'value']].rename(columns={'value': 'Emp_Change'})

if df_gdp.empty or df_emp.empty:
    print("⚠️ WARNING: Data extraction failed. Check source_file names.")

df_proxy = pd.merge(df_gdp, df_emp, on=['date', 'clean_sector'], how='inner')
df_proxy.sort_values(by=['clean_sector', 'date'], inplace=True)

if not df_proxy.empty:
    # Robust productivity calculation
    df_proxy['Emp_Index'] = df_proxy.groupby('clean_sector')['Emp_Change'].transform(lambda x: x.cumsum() - x.cumsum().min() + 100)
    df_proxy['Labor_Productivity'] = df_proxy['GDP_Nominal'] / df_proxy['Emp_Index']

# --- B. The Master Merge ---
df_proxy.rename(columns={'clean_sector': 'mapped_sector'}, inplace=True)
df_proxy['date'] = pd.to_datetime(df_proxy['date'])
df_eri['date'] = pd.to_datetime(df_eri['date'])

df_model = pd.merge(df_eri, df_proxy[['mapped_sector', 'date', 'Labor_Productivity']], on=['mapped_sector', 'date'], how='inner')

if 'Archetype' not in df_model.columns:
    df_model = pd.merge(df_model, df_meta[['mapped_sector', 'Archetype']], on='mapped_sector', how='left')

# --- C. Advanced Features ---
df_model.sort_values(by=['mapped_sector', 'date'], inplace=True)
df_model['Prod_Lag2Y'] = df_model.groupby('mapped_sector')['Labor_Productivity'].shift(8)
df_model['Prod_Centered'] = df_model['Labor_Productivity'] - df_model['Labor_Productivity'].mean()
df_model['Prod_Squared'] = df_model['Prod_Centered'] ** 2
# Ensure fillna uses the new label format
df_model['Archetype'] = df_model['Archetype'].fillna('Distressed\n(Low Res, High Vol)')

print(f">> Master Dataset Ready. Shape: {df_model.shape}")

# ==========================================
# 2. STATISTICAL INFERENCE (THE "WHY")
# ==========================================
print("\n>> RUNNING STATISTICAL INFERENCE...")

if not df_model.empty:
    df_panel = df_model.set_index(['mapped_sector', 'date'])
    try:
        mod_fe = PanelOLS.from_formula('ERI_Score_V1 ~ Labor_Productivity + EntityEffects', data=df_panel)
        res_fe = mod_fe.fit(cov_type='clustered', cluster_entity=True)
        print("--- PANEL FIXED EFFECTS (V1) ---")
        print(res_fe.summary.tables[1])
    except:
        print("⚠️ PanelOLS Failed.")

# ==========================================
# 3. MACHINE LEARNING TOURNAMENT (THE "HOW")
# ==========================================
def run_tournament(target_col, data_df):
    print(f"\n>> RUNNING TOURNAMENT FOR TARGET: {target_col}")

    # Deterministic Sort
    df_clean = data_df.dropna(subset=[target_col, 'Labor_Productivity', 'Archetype']).sort_values(by=['mapped_sector', 'date'])
    
    if df_clean.empty: return None, "None", 0.0

    X = df_clean[['Labor_Productivity', 'Archetype']]
    y = df_clean[target_col]

    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), ['Labor_Productivity']),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), ['Archetype'])
    ])

    models = {
        'Ridge (Linear)': Pipeline([('prep', preprocessor), ('reg', Ridge())]),
        'Poly (Deg 2)': Pipeline([('prep', preprocessor), ('poly', PolynomialFeatures(2)), ('reg', LinearRegression())]),
        'Random Forest': Pipeline([('prep', preprocessor), ('reg', RandomForestRegressor(n_estimators=200, random_state=42))]),
        'Gradient Boosting': Pipeline([('prep', preprocessor), ('reg', GradientBoostingRegressor(n_estimators=200, random_state=42))])
    }

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    results = []

    for name, model in models.items():
        try:
            model.fit(X_train, y_train)
            score = model.score(X_test, y_test)
            results.append({'Model': name, 'R2': score})
        except:
            results.append({'Model': name, 'R2': -np.inf})

    df_res = pd.DataFrame(results).sort_values('R2', ascending=False)
    print(df_res)
    best_row = df_res.iloc[0]
    return models[best_row['Model']], best_row['Model'], best_row['R2']

champ_v1, name_v1, score_v1 = run_tournament('ERI_Score_V1', df_model)
champ_v2, name_v2, score_v2 = run_tournament('ERI_Score_V2', df_model)

# ==========================================
# 4. STRUCTURAL AUDIT (PCA & LAGS)
# ==========================================
print("\n>> RUNNING TWIN TEST (Lags & Limits)...")
twin_res = []
df_twin = df_model.dropna(subset=['Prod_Lag2Y', 'Prod_Squared'])

if not df_twin.empty:
    for target in ['ERI_Score_V1', 'ERI_Score_V2']:
        try:
            mod_lag = smf.ols(f"{target} ~ Prod_Lag2Y + C(Archetype)", data=df_twin).fit()
            is_j_curve = "YES" if (mod_lag.params['Prod_Lag2Y'] > 0 and mod_lag.pvalues['Prod_Lag2Y'] < 0.05) else "NO"

            mod_quad = smf.ols(f"{target} ~ Prod_Centered + Prod_Squared + C(Archetype)", data=df_twin).fit()
            is_limit = "YES" if (mod_quad.params['Prod_Squared'] < 0 and mod_quad.pvalues['Prod_Squared'] < 0.05) else "NO"

            twin_res.append({'Index': target, 'J-Curve (Lag)': is_j_curve, 'Eff Limit (Quad)': is_limit})
        except:
            twin_res.append({'Index': target, 'J-Curve (Lag)': 'ERR', 'Eff Limit (Quad)': 'ERR'})

    df_twin_res = pd.DataFrame(twin_res)
    print(df_twin_res)
else:
    print("⚠️ Skipping Twin Test: Not enough data.")
    df_twin_res = pd.DataFrame()

# ==========================================
# 5. FORECASTING (2026: SIDE-BY-SIDE)
# ==========================================
print("\n>> RUNNING 2026 STRESS TEST (V1 vs V2)...")

if champ_v1 and champ_v2 and not df_model.empty:
    latest_data = df_model.sort_values('date').groupby('mapped_sector').tail(1).copy()
    scenarios = {'Shock (-10%)': 0.90, 'Baseline': 1.00, 'Boom (+10%)': 1.10}

    sim_results = []
    for sc_name, multiplier in scenarios.items():
        sim_df = latest_data.copy()
        sim_df['Labor_Productivity'] *= multiplier
        sim_df['Pred_V1'] = champ_v1.predict(sim_df[['Labor_Productivity', 'Archetype']])
        sim_df['Pred_V2'] = champ_v2.predict(sim_df[['Labor_Productivity', 'Archetype']])
        agg = sim_df.groupby('Archetype')[['Pred_V1', 'Pred_V2']].mean().reset_index()
        agg['Scenario'] = sc_name
        sim_results.append(agg)

    if sim_results:
        df_sim = pd.concat(sim_results)

        fig, axes = plt.subplots(1, 2, figsize=(18, 7), sharey=True)
        
        # We manually update ticks to avoid UserWarning
        def set_labels(ax):
            ticks = ax.get_xticks()
            labels = ax.get_xticklabels()
            # If labels are empty (sometimes happens before drawing), we skip wrapping
            if len(labels) > 0:
                ax.set_xticks(ticks) # Fix ticks
                ax.set_xticklabels(labels, rotation=0)

        # Plot V1
        sns.barplot(
            data=df_sim, x='Archetype', y='Pred_V1', hue='Scenario',
            palette={'Shock (-10%)': '#d62728', 'Baseline': 'gray', 'Boom (+10%)': '#1f77b4'},
            ax=axes[0]
        )
        axes[0].set_title(f'V1 Forecast: Policy View\n(Sensitive)\nChampion: {name_v1}', fontsize=12)
        axes[0].set_ylabel('Projected Resilience Score')
        axes[0].set_ylim(0.4, 0.85)
        axes[0].grid(axis='y', linestyle='--', alpha=0.3)
        if axes[0].get_legend(): axes[0].get_legend().remove()
        set_labels(axes[0])

        # Plot V2
        sns.barplot(
            data=df_sim, x='Archetype', y='Pred_V2', hue='Scenario',
            palette={'Shock (-10%)': '#d62728', 'Baseline': 'gray', 'Boom (+10%)': '#1f77b4'},
            ax=axes[1]
        )
        axes[1].set_title(f'V2 Forecast: Robust View\n(Stable/Stubborn)\nChampion: {name_v2}', fontsize=12)
        axes[1].set_xlabel('Sector Archetype')
        axes[1].grid(axis='y', linestyle='--', alpha=0.3)
        axes[1].legend(title='2026 Scenario', bbox_to_anchor=(1.02, 1), loc='upper left')
        set_labels(axes[1])

        plt.tight_layout()
        plt.savefig(os.path.join(IMG_DIR, '10_Forecast_Comparison_2026.png'), dpi=300)
        plt.show()

        # Grand Comparison Table
        print("\n>> SAVING GRAND COMPARISON...")
        v1_lag = df_twin_res.iloc[0]['J-Curve (Lag)'] if not df_twin_res.empty else "N/A"
        v2_lag = df_twin_res.iloc[1]['J-Curve (Lag)'] if not df_twin_res.empty else "N/A"
        
        grand_comp = pd.DataFrame({
            'Metric': ['Champion Model', 'Fit (R2)', 'Lag Effect'],
            'V1 (Policy)': [name_v1, f"{score_v1:.2f}", v1_lag],
            'V2 (Robust)': [name_v2, f"{score_v2:.2f}", v2_lag]
        })
        grand_comp.to_csv(os.path.join(OUTPUT_DIR, 'ERI_Phase4_Grand_Comparison.csv'), index=False)
        print(grand_comp)

print("\n>> PHASE 4 COMPLETE.")